In [100]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, cross_val_score

import statsmodels.api as sm

In [101]:
train = pd.read_csv('../cleaned_datasets/train.csv')
test = pd.read_csv('../cleaned_datasets/test.csv')

***
## Feature Engineering
***

In [102]:
# combining Train and Test for feature engineering
whole = pd.concat([train, test], sort = False)


# dropping price so no columns contain direct price information
whole.drop(columns = 'saleprice', inplace = True)


# with pd.option_context('display.max_rows', None):

#     display(whole.corr())

corrs = whole.corr()



for i, col in enumerate(corrs):
    
    corrs[col] = corrs[col][i+1:]
    
    
corrs = corrs.stack()

In [103]:

correlated = []

for key, val in corrs.iteritems():
    
    if abs(val) >= .5:
        
        correlated.append(key)


        


temp = list(zip(*correlated))[0] + list(zip(*correlated))[1]


[val for val in temp if val not in whole.columns]

[]

In [104]:
# making features and concating to whole



features = pd.DataFrame(columns = range(1))


for val in correlated:
    features[f'{val[0]}_{val[1]}'] = whole[val[0]] * whole[val[1]]
        



whole = pd.concat([whole, features], axis= 1)



In [105]:
whole.drop(columns= ['index', 0], inplace= True)

whole

,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,garage_finish_Unf_year_built,garage_finish_Unf_garage_type_Detchd,garage_finish_Unf_garage_finish_RFn,garage_qual_TA_garage_qual_Fa,garage_cond_TA_garage_qual_TA,garage_cond_TA_garage_cond_Fa,pool_qc_Gd_pool_area,misc_feature_Gar2_misc_val,misc_feature_TenC_pool_qc_Fa,sale_type_WD _sale_type_New
0,109.0,533352170.0,60.0,57.480147,13517.0,6.0,8.0,1976.0,2005.0,289.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,544.0,531379050.0,60.0,43.000000,11492.0,7.0,5.0,1996.0,1997.0,132.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,153.0,535304180.0,20.0,68.000000,7922.0,5.0,7.0,1953.0,2007.0,0.0,...,1953.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,318.0,916386060.0,60.0,73.000000,9802.0,5.0,5.0,2006.0,2007.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,255.0,906425045.0,50.0,82.000000,14235.0,6.0,8.0,1900.0,1993.0,0.0,...,1900.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662.0,527377110.0,60.0,80.000000,8000.0,6.0,6.0,1974.0,1974.0,0.0,...,1974.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
874,1234.0,535126140.0,60.0,90.000000,14670.0,6.0,7.0,1966.0,1999.0,410.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
875,1373.0,904100040.0,20.0,55.000000,8250.0,5.0,5.0,1968.0,1968.0,0.0,...,1968.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
876,1672.0,527425140.0,20.0,60.000000,9000.0,4.0,6.0,1971.0,1971.0,0.0,...,1971.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [106]:


train_ = whole.iloc[:2051]
test_ = whole.iloc[2051:]




train_['saleprice'] = train['saleprice']
test_['saleprice'] = test['saleprice']




train = train_
test = test_






C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [107]:
X = train.drop(columns='saleprice')
y = train['saleprice']
X = sm.add_constant(X)



C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [108]:

model = sm.OLS(y, X).fit()
high_p_vals = list(model.pvalues[model.pvalues >= .05].index)[2:]
# X.drop(columns = high_p_vals, inplace = True)


X.drop(columns = 'const', inplace=True)

C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [109]:
len(high_p_vals)

269

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)



In [111]:
lr = LinearRegression()

y_train_log = y_train.map(np.log)
y_test_log = y_test.map(np.log)



print(cross_val_score(lr, X_train, y_train_log, cv=5).mean()
)




lr.fit(X_train, y_train_log)

lr.score(X_train, y_train_log), lr.score(X_test, y_test_log)




-1.0183799944449452


(0.9613499950138943, 0.8330568740951068)

In [112]:
lr.fit(X, y.map(np.log))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [113]:


testing = test.drop(columns = 'saleprice')

# testing.drop(columns = high_p_vals, inplace = True)

display(testing)

output = pd.DataFrame({'Id': test['id']})


output['SalePrice'] = lr.predict(testing)


output['SalePrice'] = output['SalePrice'].map(np.exp)

output['Id'] = [int(val) for val in output['Id']]

display(output)

pd.DataFrame.to_csv(output, '../submissions/feat_engineer_correlation_based.csv', index = False)

,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,garage_finish_Unf_year_built,garage_finish_Unf_garage_type_Detchd,garage_finish_Unf_garage_finish_RFn,garage_qual_TA_garage_qual_Fa,garage_cond_TA_garage_qual_TA,garage_cond_TA_garage_cond_Fa,pool_qc_Gd_pool_area,misc_feature_Gar2_misc_val,misc_feature_TenC_pool_qc_Fa,sale_type_WD _sale_type_New
0,2658.0,902301120.0,190.0,69.000000,9142.0,6.0,8.0,1910.0,1950.0,0.0,...,1910.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2718.0,905108090.0,90.0,82.786765,9662.0,5.0,4.0,1977.0,1977.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2414.0,528218130.0,60.0,58.000000,17104.0,7.0,5.0,2006.0,2006.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1989.0,902207150.0,30.0,60.000000,8520.0,5.0,6.0,1923.0,2006.0,0.0,...,1923.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,625.0,535105100.0,20.0,79.027933,9500.0,6.0,5.0,1963.0,1963.0,247.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662.0,527377110.0,60.0,80.000000,8000.0,6.0,6.0,1974.0,1974.0,0.0,...,1974.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
874,1234.0,535126140.0,60.0,90.000000,14670.0,6.0,7.0,1966.0,1999.0,410.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
875,1373.0,904100040.0,20.0,55.000000,8250.0,5.0,5.0,1968.0,1968.0,0.0,...,1968.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
876,1672.0,527425140.0,20.0,60.000000,9000.0,4.0,6.0,1971.0,1971.0,0.0,...,1971.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


,Id,SalePrice
0,2658,95375.512295
1,2718,155469.278019
2,2414,206150.787217
3,1989,97663.889820
4,625,172125.328548
...,...,...
873,1662,201651.409024
874,1234,206321.020273
875,1373,130054.634142
876,1672,115481.208723
